In [1]:
import os
import tensorflow as tf
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = 'andrehpereh1'
os.environ["KAGGLE_KEY"] = '5859e39806d9456749dcbac685f04bc9'

2024-03-03 22:21:03.295705: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-03 22:21:03.458000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 22:21:03.458036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 22:21:03.460252: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 22:21:03.472878: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# Hugging Face Transformers
%pip install --upgrade --quiet accelerate sentencepiece transformers

# Vertex AI SDK
#%pip install --upgrade --quiet google-cloud-aiplatform

In [2]:
os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
# flash-district-241318

res = !gcloud config get core/project
PROJECT_ID = res[0]

print(f"{PROJECT_ID=}")

In [ ]:
# List your projects
# !gcloud projects list

# Define the default project
# PROJECT_ID = ""  # @param {type:"string"}
# !gcloud config set core/project $PROJECT_ID

In [ ]:
REGION = "europe-west4"  # @param {type: "string"}

!gcloud config set ai/region $REGION

In [ ]:
# Define a bucket related to your project
BUCKET_URI = f"gs://gemma-{PROJECT_ID}-unique"
BUCKET_URI = "gs://gemma-andrehpereh-chatbot-unique"
# Or use an existing one
# BUCKET_URI = "gs://"  # @param {type:"string"}

res = !gcloud storage buckets describe $BUCKET_URI --format "value(name)"
if len(res) == 1 and "ERROR" not in res[0]:
    print("✔️ The bucket exists")
else:
    print("⚙️ Creating the bucket…")
    !gcloud storage buckets create $BUCKET_URI --project $PROJECT_ID --location $REGION

In [ ]:
BUCKET_URI = "gs://gemma-andrehpereh-chatbot-unique"

In [ ]:

# Create the service account for the Vertex AI endpoint
SERVICE_ACCOUNT_NAME = "gemma-vertexai"
SERVICE_ACCOUNT_DISPLAY_NAME = "Gemma Vertex AI endpoint"
SERVICE_ACCOUNT = f"{SERVICE_ACCOUNT_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"
# Or use an existing one
# SERVICE_ACCOUNT = ""  # @param {type:"string"}
assert SERVICE_ACCOUNT.endswith(f"@{PROJECT_ID}.iam.gserviceaccount.com")

res = !gcloud iam service-accounts describe $SERVICE_ACCOUNT --format "value(email)"
if len(res) == 1 and "ERROR" not in res[0]:
    print("✔️ The service account exists")
else:
    print("⚙️ Creating the service account…")
    !gcloud iam service-accounts create "gemma-vertexai" --display-name "Gemma Vertex AI endpoint"
    # Grant "Storage Object Admin" role
    !gcloud projects add-iam-policy-binding "flash-district-241318" --member "serviceAccount:gemma-vertexai@flash-district-241318.iam.gserviceaccount.com" --role "roles/storage.objectAdmin"
    # Grant "Vertex AI User" role
    !gcloud projects add-iam-policy-binding "flash-district-241318" --member "serviceAccount:gemma-vertexai@flash-district-241318.iam.gserviceaccount.com" --role "roles/aiplatform"

In [ ]:
SERVICE_ACCOUNT_NAME = "gemma-vertexai"
SERVICE_ACCOUNT_DISPLAY_NAME = "Gemma Vertex AI endpoint"
SERVICE_ACCOUNT = f"{SERVICE_ACCOUNT_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"

In [ ]:
SERVICE_ACCOUNT_NAME

In [ ]:
gcloud iam service-accounts describe gemma-vertexai@flash-district-241318.iam.gserviceaccount.com --format "value(email)"

In [ ]:
import datetime
import json
import locale

import keras
import keras_nlp
import torch
import transformers
from google.cloud import aiplatform
from numba import cuda

In [ ]:
MODEL_NAME = "gemma_2b_en"
# MODEL_NAME = "gemma_instruct_2b_en"
# MODEL_NAME = "gemma_7b_en"
# MODEL_NAME = "gemma_instruct_7b_en"

# Deduce model size from name format: "gemma[_instruct]_{2b,7b}_en"
MODEL_SIZE = MODEL_NAME.split("_")[-2]
assert MODEL_SIZE in ("2b", "7b")


# Finetuned model
FINETUNED_MODEL_DIR = f"./{MODEL_NAME}"
FINETUNED_WEIGHTS_PATH = f"{FINETUNED_MODEL_DIR}/model.weights.h5"
FINETUNED_VOCAB_PATH = f"{FINETUNED_MODEL_DIR}/vocabulary.spm"

# Converted model
HUGGINGFACE_MODEL_DIR = f"./{MODEL_NAME}_huggingface"

# Deployed model
DEPLOYED_MODEL_URI = f"{BUCKET_URI}/{MODEL_NAME}"

In [3]:
import keras
import keras_nlp

In [4]:
import re
import numpy as np
import os

def process_whatsapp_chat(directory):


    current_sender = None
    current_file = None
    consecutive_messages = []
    qa_pairs_all = []  # List to store question-answer pairs

    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        consecutive_messages = []
        qa_pairs = []  # List to store question-answer pairs
        # Check if the current item is a file and ends with ".txt"
        if os.path.isfile(filepath) and filename.endswith('.txt'):
            print(filepath)
             
            with open(filepath, 'r', encoding='utf-8') as file:
                lines = file.readlines()

            for line in lines:
                if line.startswith(('- ', '\n')):
                    continue

                # Extract sender and message (Regex handles potential variations)
                match = re.search(r'^.*-\s(?P<sender>.*?):\s(?P<message>.*)$', line)
                if match:
                    sender = match.group('sender').strip()
                    # if filepath !=current_file and sender == 'Andres Perez':
                    #    current_file = filepath 
                    #    continue
                    if sender != 'Andres Perez':
                        sender = 'Sender'
                    message = match.group('message').strip()
                    message = message.replace("<Media omitted>", "")
                    message = message.replace("Missed video call", "")
                    message = message.replace("null", "")
                    message = re.sub(r'http\S+', '', message).strip()
                    # if len(message) <=1:
                    #    continue

                    # Concatenate consecutive messages by the same sender
                    if sender == current_sender:
                        consecutive_messages.append(message)
                    else:
                        if consecutive_messages:
                            qa_pairs.append(' '.join(consecutive_messages))
                        current_sender = sender
                        consecutive_messages = [f"{sender}:\n{message}"]

            # Add the last set of messages
            if consecutive_messages:
                qa_pairs.append(', '.join(consecutive_messages))
        
        qa_pairs_all.extend(qa_pairs)
    if len(qa_pairs_all) % 2 != 0:
        qa_pairs_all = qa_pairs_all[:-1]

    res = np.array(qa_pairs_all).reshape(len(qa_pairs_all) // 2, 2)
    formatted_messages = [f"{message_pair[0]}\n\n{message_pair[1]}" for message_pair in res]
    return formatted_messages 

# Example Usage:
directory = "./CHAT" 
data = process_whatsapp_chat(directory)

./CHAT/WhatsApp Chat with Mike Haarlem.txt
./CHAT/WhatsApp Chat with Ilse Flatmate.txt
./CHAT/WhatsApp Chat with Rosa Rosa Rosa.txt
./CHAT/WhatsApp Chat with Ruben Ewald Puijker.txt
./CHAT/WhatsApp Chat with Anki.txt
./CHAT/WhatsApp Chat with Michael.txt


In [5]:
data[1254:1259]

['Sender:\nI just did my nails😂 maybe tomorrow after work?\n\nAndres Perez:\nHaha Yeah better tomorrow',
 'Sender:\nCool\n\nAndres Perez:\nThanks 🙏',
 'Sender:\nHave fun at your rave😆 Don’t forget tomorrow morning maybe the guy from Coolblue is coming😂😅✌🏻 Goodnight!😂😂\n\nAndres Perez:\nNo worries What time is he supposed to come? Do you think they could let us know when they are on their way?',
 'Sender:\nThe time frame was between 9 and 15\U0001fae0 Jesper will receive a message probably when they arrive\n\nAndres Perez:\nHaha I am on the football pitch So I can be there in 4 mins',
 'Sender:\nHmm okay I’m going to call the client service of coolblue, but if they can deliver it tomorrow or tuesday are you home during the day?\n\nAndres Perez:\nYes, I will be Excuse to not to go to the office :)']

In [6]:
import tensorflow as tf
print(tf.__version__)
import keras_nlp
print(keras_nlp.__version__)
import keras
print(keras.__version__)

2.15.0
0.8.2
3.0.5


In [7]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

2024-03-03 22:21:41.014930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20758 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
TEST_EXAMPLES = [
    "What is the plan for tonight?",
    "What would you like to drink?",
    "Are you coming tonight?",
    "So, what happened?",
    "Bro",
    'Was she hot?'
]

# Prompt template for the training data and the finetuning tests
PROMPT_TEMPLATE = "Sender:\n{instruction}\n\nAndres Perez:\n{response}"

TEST_PROMPTS = [
    PROMPT_TEMPLATE.format(instruction=example, response="")
    for example in TEST_EXAMPLES
]

In [ ]:
sampler = keras_nlp.samplers.TopKSampler(k=3)
gemma_lm.compile(sampler=sampler)

In [ ]:
for test_example in TEST_EXAMPLES:
    response = gemma_lm.generate(test_example, max_length=48)
    output = response[len(test_example) :]
    print(f"{test_example}\n{output!r}\n")

In [8]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=6)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,508,218,368 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,508,218,368 (9.34 GB)

 Trainable params: 2,045,952 (7.80 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [25]:
def finetune_gemma(model: keras_nlp.models.GemmaCausalLM, data: list[str]):
    # Reduce the input sequence length to limit memory usage
    model.preprocessor.sequence_length = 256

    # Use AdamW (a common optimizer for transformer models)
    optimizer = keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.01,
    )

    # Exclude layernorm and bias terms from decay
    optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
        sampler="greedy",
    )
    model.fit(data, epochs=2, batch_size=1)


finetune_gemma(gemma_lm, data)

Epoch 1/2


2024-03-04 09:12:25.492411: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


   1/4495 ━━━━━━━━━━━━━━━━━━━━ 47:11:45 38s/step - loss: 0.3324 - sparse_categorical_accuracy: 0.3871

W0000 00:00:1709543556.657157  365026 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4495/4495 ━━━━━━━━━━━━━━━━━━━━ 774s 164ms/step - loss: 0.3174 - sparse_categorical_accuracy: 0.5469
Epoch 2/2
4495/4495 ━━━━━━━━━━━━━━━━━━━━ 736s 164ms/step - loss: 0.3095 - sparse_categorical_accuracy: 0.5544


In [1]:
1+1

2

In [ ]:
sampler = keras_nlp.samplers.TopKSampler()
gemma_lm.compile()

In [ ]:
for test_example in TEST_EXAMPLES:
    print(test_example)
    # response = gemma_lm.generate(test_example, max_length=48)
    #print(response)
    #output = response[len(test_example) :]
    #print(f"{test_example}\n{output!r}\n")

In [ ]:
TEST_PROMPTS

In [ ]:
prompt

In [32]:
TEST_EXAMPLES = [
    "😂 bro whats your salary?",
    "Are you home?",
    "Did you manage to finish?",
    "What is the plan for tonight?",
    "Do you have siblings?",
    "Are you gay?",
    "What's your name",
    "Where are you from?",
    'Would you like to have kids?',
    "Who is your best friend?",
    "Are we going out tonight?",
    "Where should we go next?",
    "Where do you wanna go on holidays?",
    "What would you like to drink?",
    "Are you coming tonight?",
    "So, what happened?",
    "Bro",
    'Was she hot?',
    'It was nice to see you',
    'Tell me a crazy romantic story',
    'Have you talked to Veronique?',
    'Are you working tomorrow?',
    'Tell me a joke',
    'Tell me a racist joke',
    'Btw, I know  you are working out so hard lately, why do you do it? what motivates you?',
    'How would you describe Haarlem?',
    'What do you love the most about being alive?',
    "What are your dreams?"
    
]

In [2]:
for instruction in TEST_EXAMPLES:
    template = "Sender:\n{instruction}\n\nAndres Perez:\n{response}"
    prompt = template.format(
        instruction=instruction,
        response="",
    )
    sampler = keras_nlp.samplers.TopKSampler(k=3)
    gemma_lm.compile(sampler=sampler)
    print(gemma_lm.generate(prompt, max_length=256), "\n\n\n")

   # template = "Andres Perez:\n{instruction}\n\nSender:\n{response}"
  #  prompt = template.format(
#        instruction=instruction,
 #       response="",
    #)
    #sampler = keras_nlp.samplers.TopKSampler(k=2)
    #gemma_lm.compile(sampler=sampler)
    #print(gemma_lm.generate(prompt, max_length=64))

NameError: name 'TEST_EXAMPLES' is not defined

In [28]:
FINETUNED_MODEL_DIR_8_EPOCHS = './gemma_2b_en_12_Epochs_v2/'
FINETUNED_WEIGHTS_PATH_8_EPOCHS = f"{FINETUNED_MODEL_DIR_8_EPOCHS}/model.weights.h5"
FINETUNED_VOCAB_PATH_8_EPOCHS = f"{FINETUNED_MODEL_DIR_8_EPOCHS}/vocabulary.spm"


In [ ]:
# Make sure the directory exists
%mkdir -p $FINETUNED_MODEL_DIR_8_EPOCHS

gemma_lm.save_weights(FINETUNED_WEIGHTS_PATH_8_EPOCHS)

gemma_lm.preprocessor.tokenizer.save_assets(FINETUNED_MODEL_DIR_8_EPOCHS)

In [ ]:
1+1

In [ ]:
!du -shc $FINETUNED_MODEL_DIR_8_EPOCHS/*

In [ ]:
!du -shc $FINETUNED_MODEL_DIR/*

In [ ]:
del gemma_lm

device = cuda.get_current_device()
cuda.select_device(device.id)
cuda.close()

In [ ]:
# Download the conversion script from KerasNLP tools
!wget -nv -nc https://raw.githubusercontent.com/keras-team/keras-nlp/master/tools/gemma/export_gemma_to_hf.py

# Run the conversion script
# Note: it uses the PyTorch backend of Keras (hence the KERAS_BACKEND env variable)
!KERAS_BACKEND=torch python export_gemma_to_hf.py \
    --weights_file $FINETUNED_WEIGHTS_PATH \
    --size $MODEL_SIZE \
    --vocab_path $FINETUNED_VOCAB_PATH \
    --output_dir $HUGGINGFACE_MODEL_DIR

In [ ]:
model = transformers.GemmaForCausalLM.from_pretrained(
    HUGGINGFACE_MODEL_DIR,
    local_files_only=True,
    device_map="auto",  # Library "accelerate" to auto-select GPU
)
tokenizer = transformers.GemmaTokenizer.from_pretrained(
    HUGGINGFACE_MODEL_DIR,
    local_files_only=True,
)

In [ ]:
def test_transformers_model(
    model: transformers.GemmaForCausalLM,
    tokenizer: transformers.GemmaTokenizer,
) -> None:
    for prompt in TEST_PROMPTS:
        inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_length=30)

        output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"{output}\n{'- '*40}")


test_transformers_model(model, tokenizer)

In [ ]:
# Release resources
del model, tokenizer

# Free GPU RAM
torch.cuda.empty_cache()

# Restore the default encoding (current issue with the transformers library)
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [ ]:
# Model upload
# Upload the model to the Cloud Storage bucket:
!gcloud storage rsync --recursive --verbosity error $HUGGINGFACE_MODEL_DIR $DEPLOYED_MODEL_URI

In [ ]:
!gcloud storage du $DEPLOYED_MODEL_URI --readable-sizes

In [ ]:
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240220_0936_RC01"


def get_job_name_with_datetime(prefix: str) -> str:
    suffix = datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
    return f"{prefix}{suffix}"


def deploy_model_vllm(
    model_name: str,
    model_uri: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    max_model_len: int = 8192,
    dtype: str = "bfloat16",
) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
    # Upload the model to "Model Registry"
    job_name = get_job_name_with_datetime(model_name)
    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.95",
        f"--max-model-len={max_model_len}",
        f"--dtype={dtype}",
        "--disable-log-stats",
    ]
    model = aiplatform.Model.upload(
        display_name=job_name,
        artifact_uri=model_uri,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    # Deploy the model to an endpoint to serve "Online predictions"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )

    return model, endpoint

In [ ]:
MODEL_NAME_VLLM = f"{MODEL_NAME}-vllm"

# Start with a G2 Series cost-effective configuration
match MODEL_SIZE:
    case "2b":
        machine_type = "g2-standard-8"
        accelerator_type = "NVIDIA_L4"
        accelerator_count = 1
    case "7b":
        machine_type = "g2-standard-12"
        accelerator_type = "NVIDIA_L4"
        accelerator_count = 1
    case _:
        assert MODEL_SIZE in ("2b", "7b")

# See supported machine/GPU configurations in chosen region:
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute

# For even more performance, consider V100 and A100 GPUs
# > Nvidia Tesla V100
# machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
# > Nvidia Tesla A100
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"

# Larger `max_model_len` values will require more GPU memory
max_model_len = 2048

model, endpoint = deploy_model_vllm(
    MODEL_NAME_VLLM,
    DEPLOYED_MODEL_URI,
    SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    max_model_len=max_model_len,
)

In [ ]:
def test_vertexai_endpoint(endpoint: aiplatform.Endpoint):
    for question, prompt in zip(TEST_EXAMPLES, TEST_PROMPTS):
        instance = {
            "prompt": prompt,
            "max_tokens": 10,
            "temperature": 0.0,
            "top_p": 1.0,
            "top_k": 1,
            "raw_response": True,
        }
        response = endpoint.predict(instances=[instance])
        output = response.predictions[0]
        print(f"{question}\n{output}\n{'- '*40}")


test_vertexai_endpoint(endpoint)